In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
ss = StandardScaler()

In [6]:
scaler_housing_with_bias = np.c_[np.ones(shape=(housing.data.shape[0], 1)), ss.fit_transform(housing.data)]

In [7]:
housing_labels = housing.target.reshape(-1, 1)

In [8]:
num_examples, num_features = scaler_housing_with_bias.shape

In [9]:
lr = 0.005
batch_size = 100
num_epochs = 500
num_batches = int(np.ceil(num_epochs/batch_size))

In [10]:
def get_batch(idx_epoch, idx_batch, batch_size):
    np.random.seed(idx_epoch * num_batches + idx_batch )
    indexes = np.random.randint(low=0, high=num_examples, size=batch_size)
    x_batch = scaler_housing_with_bias[indexes]
    y_batch = housing_labels[indexes]
    return x_batch, y_batch

In [11]:
with tf.name_scope("my_graph") as scope:
    x = tf.placeholder(dtype=tf.float32, shape=(None, num_features))
    y = tf.placeholder(dtype=tf.float32, shape=(None, 1))
    theta = tf.Variable(tf.random_normal((num_features, 1)), dtype=tf.float32)
    y_pred = tf.matmul(x, theta)
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    training_op = optimizer.minimize(mse)

In [12]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_dir = 'logs'
log_dir = "{0}/runa-{1}".format(root_dir, now)


mse_summary = tf.summary.scalar("MSE", mse)
file_summary = tf.summary.FileWriter(log_dir, tf.get_default_graph())

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [13]:
with tf.Session(config=config) as sess:
    sess.run(init)
    
    for e in range(num_epochs):
        for b in range(num_batches):
            x_batch, y_batch = get_batch(e, b, batch_size)
            feed = {x: x_batch, y: y_batch}
            sess.run(training_op, feed_dict=feed)
        saver.save(sess, '/tmp/model/my_model2.ckpt')
        summ = sess.run(mse_summary, {x: scaler_housing_with_bias, y: housing_labels})
        step = e
        file_summary.add_summary(summ, step)
    
    saver.save(sess, '/tmp/model/my_model2-final.ckpt')
    best_weights = theta.eval()
    file_summary.close()